<a href="https://colab.research.google.com/github/ejjjjkang/computational_methods/blob/main/hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INFO 6010, Homework 3



In [ ]:
## Run this cell ONCE to install the simpledorff package.
## Remove the `# ` below to uncomment the pip install command.
## Lines that start with `!` are sent to the operating system, not python itself.

!pip install simpledorff

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

## you may need to install this package
import simpledorff

from collections import Counter

from scipy.stats import ttest_1samp, norm, t

## Problem 1: sample size calculation ( points)

This problem reviews work from the previous homework. A two-sample t-test evaluates whether the difference in means between two arrays is significant. The important factors are (a) the difference between the means, (b) the sample sizes in the two arrays, and (c) the variability of the values in the arrays, relative to the difference between means. The *effect size*  is the difference between means expressed as a multiple of the standard deviation of the arrays. (We'll assume both arrays have the same std. dev.)

Previously we were choosing an effect size and sample size and measuring the proportion of p-values below some threshold. What if instead we choose the effect size and the proportion of p-values below a threshold, and vary the sample size? In other words, how many samples do we need to detect an effect of size $d$ with probability $\alpha$? This is a *power calculation*.

Wikipedia has a helpful table on the page for [sample size determination](https://en.wikipedia.org/wiki/Sample_size_determination) (look for *Cohen's d*) that shows the appropriate minimum sample size for different effect sizes and *power* values $\alpha$ at $p<0.05$. For example, to detect a difference of 0.2 standard deviations with 80% probability at 0.05, you would need 393 samples, whereas a 0.5 difference would only need 64.

Use a simulation to replicate this result. For sample size $N$ from 5-100 in increments of 5, do the following experiment 1000 times: sample an array `a` of size $N$ from a normal distribution with mean 0.0 and SD 1.0, and an array `b` of size $N$ from a normal distribution with mean 0.5 and SD 1.0. Then run a two-sample t-test comparing these two arrays, and record the p-value. Print $N$ and the number of p-values less than 0.05.



In [ ]:
from scipy.stats import ttest_ind

sam_size = np.arange(5, 100, 5)

num_of_pvalue_per_sample = []
for m in sam_size:
  pvalue = []
  for i in range(1000):
    a = norm.rvs(loc=0, scale=1, size=m)
    b = norm.rvs(loc=0.5, scale=1.0, size= m)
    n = ttest_ind(a, b)
    # effect_size = (np.mean(a) -np.mean(b)) / np.std(a)
    if n.pvalue < 0.05:
      pvalue.append(n.pvalue)

  num_of_pvalue_per_sample.append(len(pvalue))
  print("Sample_size N: ", m)
  print("The num of p-values: ", len(pvalue))


Sample_size N:  5
The num of p-values:  101
Sample_size N:  10
The num of p-values:  194
Sample_size N:  15
The num of p-values:  297
Sample_size N:  20
The num of p-values:  322
Sample_size N:  25
The num of p-values:  418
Sample_size N:  30
The num of p-values:  469
Sample_size N:  35
The num of p-values:  534
Sample_size N:  40
The num of p-values:  593
Sample_size N:  45
The num of p-values:  644
Sample_size N:  50
The num of p-values:  716
Sample_size N:  55
The num of p-values:  738
Sample_size N:  60
The num of p-values:  769
Sample_size N:  65
The num of p-values:  806
Sample_size N:  70
The num of p-values:  840
Sample_size N:  75
The num of p-values:  845
Sample_size N:  80
The num of p-values:  865
Sample_size N:  85
The num of p-values:  894
Sample_size N:  90
The num of p-values:  935
Sample_size N:  95
The num of p-values:  935


**Answer here**

The result shows that when sample size is increased, the number of p-values is close to 1.0.


## Problem 2: Tidy data ( points)

The file `bea_state_gdp.csv` contains information about the GDP of US states from 1997-2021 from the [US Bureau of Economic Analysis](https://apps.bea.gov/itable/?ReqID=70&step=1&acrdn=1). The column headers are years and the row labels are states, suggesting that it is an "X-Y" or "wide" table. Use the pandas `melt()` function  to convert it into a "tidy" or "narrow" table.

When "melting" a wide table, we need to specify which columns provide information about the data elements in a row ("id vars"), and which provide the actual data values (everything else). In this case the id vars are `GeoFips` and `GeoName` (FIPS is a five-digit geographic code for US regions, similar to a zip code). Finally, we need to specify names for the two new columns representing `Year` and `GDP`. See the [documentation](https://pandas.pydata.org/docs/reference/api/pandas.melt.html) for the parameters to specify these column names.



In [ ]:
#this ipython notebook is ran on colab notebook

from google.colab import files

uploaded = files.upload()

Saving bea_state_gdp.csv to bea_state_gdp (2).csv


In [ ]:
state_gdp_wide = pd.read_csv("bea_state_gdp.csv")
state_gdp_wide.head()

,GeoFips,GeoName,1997,1998,1999,2000,2001,2002,2003,2004,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,0,United States,11529157.0,12045824.0,12623361.0,13138035.0,13263417.0,13488357.0,13865519.0,14399696.0,...,16253970.0,16553348.0,16932051.0,17390295.0,17680274.0,18076651.0,18609078.0,19036052.0,18509143.0,19609812.0
1,1000,Alabama,144501.2,149568.2,154900.2,157221.3,156853.2,160422.4,165134.7,176625.0,...,189245.5,191369.8,189886.3,191335.2,194283.8,196974.9,200372.6,203432.7,199880.8,209979.3
2,2000,Alaska,42211.3,41095.9,40590.5,39406.6,40958.7,42979.0,42355.3,44055.0,...,58283.6,55354.3,54188.2,54740.8,54246.6,54278.7,53327.0,53433.8,50705.2,50869.4
3,4000,Arizona,168408.8,183060.5,198699.9,208439.5,213166.2,220696.7,234065.9,244317.3,...,271440.0,273481.9,276948.9,282577.0,291275.2,303606.1,314827.5,325395.3,327178.0,347656.0
4,5000,Arkansas,82571.3,84570.8,89115.2,89871.7,89789.0,92950.5,96944.5,101733.6,...,108492.1,110752.4,111734.5,112351.0,112798.1,113850.2,115885.2,117126.2,117268.2,123347.3


In [ ]:
state_gdp_reshape = pd.melt(state_gdp_wide, id_vars=["GeoFips", "GeoName"], var_name="Year", value_name="GDP")
state_gdp_reshape.head()


,GeoFips,GeoName,Year,GDP
0,0,United States,1997,11529157.0
1,1000,Alabama,1997,144501.2
2,2000,Alaska,1997,42211.3
3,4000,Arizona,1997,168408.8
4,5000,Arkansas,1997,82571.3


## Problem 3: entropy ( points)

This problem will exercise your knowledge of probability, latex and markdown, and coding.

The formula for Shannon entropy for a discrete probability distribution $p$ is $\mathcal{H}(p) = -\sum_i p_i \log p_i$.

We often create discrete probability distibutions by counting the frequency of $K$ categories, such as words. The probability of category $i$ is then $\frac{n_i}{N}$, where $n_i$ is the frequency of category $i$ and $N$ is the total sum $\sum_i n_i$.

Use algebra to express the entropy of a distribution in terms of counts $n_i$ and and the total $N$ instead of $p_i$. Simplify as much as possible. Show your work with as many steps as you need. Note that
$$x = y$$
expressions in double dollar signs are displayed on a new line.

Now implement the function `entropy_from_counts()` which takes a single `Counter` as an argument and returns the entropy. Use `np.log2()` for log calculations.

Confidence check: the entropy for `Counter(["a", "a", "a", "b", "b", "c"])` should be $\approx1.46$

**Answer here**
- First, when we define $N$ with $n_i$ as $N$ is sum of ${\{n_1, n_2, ...}\}$, $N$ can be explained as $N= \sum n_i$ \ ($n_i$ denotes counts in each categetory.)
- Then $p_i$ denotes a probability of $n_i$ out of $N$, so $p_i$ can be described as $p_i =\frac{n_i}{\sum n_i}$
- Finally, when we apply the representations to the $p_i$ in the entrophy, the entropy explained by $n_i$ is as below:
$$- \sum_i \frac{n_i}{\sum n_i} \log (\frac{n_i}{\sum n_i})$$

In [ ]:
print(Counter(["a", "a", "a", "b", "b", "c"]))

Counter({'a': 3, 'b': 2, 'c': 1})


In [ ]:
def entropy_from_counts(arr):
  entrophy = []
  for i in arr.keys():
    N = sum(arr.values())
    p_i = arr[i] / N
    logp = np.log2(p_i)
    entrophy.append(p_i * logp)
  neg_entropy = -np.sum(entrophy)
  return neg_entropy

check_arr = Counter(["a", "a", "a", "b", "b", "c"])
print("Entropy: ", entropy_from_counts(check_arr))


Entropy:  1.4591479170272448


## Problem 4 : KL divergence

Unlike entropy, which has a finite maximum, KL divergence between two probability distributions can be infinite. Consider the three probability arrays below. Use the provided function to print the divergence between all six ordered pairs.

Discuss:
* Is this function symmetric? ie does it matter which distribution is $p$ and which is $q$?
* Under what condition can the function be infinite?

In [ ]:
def kl_divergence(p, q):
    non_zeros = np.nonzero(p)
    p = p[non_zeros]
    q = q[non_zeros]

    return np.sum(p * np.log2(p/q))

In [ ]:
prob_1 = np.array([0.1, 0.3, 0.6])
prob_2 = np.array([0.2, 0.6, 0.2])
prob_3 = np.array([0.4, 0.0, 0.6])

0.# code here
a = kl_divergence(prob_1, prob_2)
a_reverse = kl_divergence(prob_2, prob_1)
b = kl_divergence(prob_2, prob_3)
b_reverse = kl_divergence(prob_3, prob_2)
c = kl_divergence(prob_1, prob_3)
c_reverse = kl_divergence(prob_3, prob_1)

print("prob_1 vs prob_2: ", a)
print("prob_2 vs prob_1: ", a_reverse)
print("prob_2 vs prob_3: ", b)
print("prob_3 vs prob_2: ", b_reverse)
print("prob_1 vs prob_3: ", c)
print("prob_3 vs prob_1: ", c_reverse)

prob_1 vs prob_2:  0.5509775004326936
prob_2 vs prob_1:  0.4830074998557688
prob_2 vs prob_3:  inf
prob_3 vs prob_2:  1.3509775004326936
prob_1 vs prob_3:  inf
prob_3 vs prob_1:  0.8


<ipython-input-61-5775a64e3481>:6: RuntimeWarning: divide by zero encountered in true_divide
  return np.sum(p * np.log2(p/q))


**Answer here**

Discuss:
* Is this function symmetric? ie does it matter which distribution is $p$ and which is $q$?
> No, it is not symmetric, as we've seen in the results, the results show that when we calculate KL-divergence of P from $Q$ and $Q$ from P reversely, two results appear differently. The order of '$P$' and '$Q$' is important because each of them has a different role. In $KL(P||Q)$, P plays a role of data's true distribution, while $Q$ is an estimated model of distribution based on $P$.

* Under what condition can the function be infinite?
> When array (prob_2) is compared with array (prob_3) and when array (prob_1) is compared with array (prob_3) are showing infinite number in KL divergence. In that case, log2q is computed as infinitue numbers, making the overall KL-divergence result as infinite. This implies that the information will infinitely surprising in the place where the probability is 0.

## Problem 5 : Jensen-Shannon distance

As you saw, KL divergence can "blow up" under certain circumstances. A safer alternative is *Jensen-Shannon distance*. For two distributions $p$ and $q$, we create a "mean" distribution $r = \frac{p+q}{2}$ and then compute $\frac{1}{2}(KL(p||r) + KL(q||r))$. (The $||$ is a notation convention for KL divergence, it doesn't mean anything special.) Create a function `jensen_shannon(p, q)` and use it to calcuate JS distance for all six ordered pairs of the three distributions from the previous problem.

Discuss:
* Is this function symmetric? Why or why not?
* Can this function be infinite? Why or why not?

In [ ]:
# code here

def jensen_shannon(p, q):
  r = (p + q) / 2
  result = 1/2 * (kl_divergence(p, r) + kl_divergence(q, r))
  return result


In [ ]:
prob_1 = np.array([0.1, 0.3, 0.6])
prob_2 = np.array([0.2, 0.6, 0.2])
prob_3 = np.array([0.4, 0.0, 0.6])

0.# code here
a = jensen_shannon(prob_1, prob_2)
a_reverse = jensen_shannon(prob_2, prob_1)
b = jensen_shannon(prob_2, prob_3)
b_reverse = jensen_shannon(prob_3, prob_2)
c = jensen_shannon(prob_3, prob_1)
c_reverse = jensen_shannon(prob_3, prob_1)

print("prob_1 vs prob_2: ", a)
print("prob_2 vs prob_1: ", a_reverse)
print("prob_2 vs prob_3: ", b)
print("prob_3 vs prob_2: ", b_reverse)
print("prob_1 vs prob_3: ", c)
print("prob_3 vs prob_1: ", c_reverse)

prob_1 vs prob_2:  0.1245112497836531
prob_2 vs prob_1:  0.1245112497836531
prob_2 vs prob_3:  0.3999999999999999
prob_3 vs prob_2:  0.3999999999999999
prob_1 vs prob_3:  0.21951797627815944
prob_3 vs prob_1:  0.21951797627815944


**Answer here**

Discuss:
* Is this function symmetric? Why or why not?
> The result is symmetric, because it considers both directions ($Q$ with $P$ or $P$ with $Q$) by using average result of the probabilities.

* Can this function be infinite? Why or why not?
> There is no infinite result. Because the result bounds between 0 and 1. Usually in the machine learning problem, when KL divergence produces infinite numbers, the Jensen-Shannon divergence solves the issue.


## Problem 6 : inter-annotator agreement

Consider two scenarios:

* **A** 10 students are assigned to rate restaurants. Each student can choose their own selection of five restaurants. It happens that each restaurant is reviewed by three students.
* **B** 10 students are assigned to rate restaurants. Each student is assigned to a selection of five restaurants. Each restaurant is reviewed by five students.

Which inter-annotator agreement metric would be appropriate for these scenarios? Which scenario do you expect to have higher agreement? Explain each answer.

**Answer here** (metrics)


Which inter-annotator agreement metric would be appropriate for these scenarios?
*   Krippendorff's alpha. There could be another option like cohen's alpha, but cohen alpha is applicable when the data is not missing and two annotators are assigned. On the other hand, Krippendorff's alpha is less limited to the constraints of the number of annotators and data missing problem. It could be applicable to the number of variables, types of variables, and the number of rators. Therefore, when only considering the description of scenarios A and B, there could be missing data and the number of rators are 5, so Krippendorff is applicable to both cases.

Which scenario do you expect to have higher agreement? Explain each answer.
*  B is a proper strategy for inter-annotator agremment. There are two reasons for this argument. First, in A, student may have chosen either their top favorite restaurants or bad restaurants based on their experiences. Therefore, it would be hard to find an agreement between students as the perceptions are hard to be hared.
* If we choose strategy B, students could be assigned randomly to five students so it is easy to build intercoders' rules and correct their result based on the rules.




## Problem 7: inter-annotator agreement 2

You have the following scores for three annotators rating six questions on a five-point Likert scale (strongly disagree, ... , strongly agree). You suspect that the distinction between "strongly (dis)agree and (dis)agree" is not reliable. Create a `pandas` data frame using these three arrays as columns named `annotators`, `items`, and `ratings` and calculate Krippendorff's alpha using the `simpledorff` package.

Then add a new column called `ratings_3` that collapses the scores {-2,-1} to -1, and {1,2} to 1. Recalculate Krippendorff's alpha.

Look up the standard interpretations for the Krippendorff values you got. Is this change significant from that perspective? What would Reviewer 2 say?

Finally, calcuate the entropy of the full array of ratings before and after the switch (just create a `Counter` from the `ratings` field or the `ratings_3` field and use your function from before).

In [ ]:
annotators = [1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3]
items = [1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6]
ratings = [-2, -1, -1, 1, 2, 1, -1, 0, -1, -2, 1, 2, 2, 1, 2, -1, -1, -2]

# create dataframe and calculate alpha here
df = pd.DataFrame({"annotators": annotators , "items": items, "ratings": ratings})
df.head()


,annotators,items,ratings
0,1,1,-2
1,2,1,-1
2,3,1,-1
3,1,2,1
4,2,2,2


In [ ]:
#calculate Krippendorff's alpha
a = simpledorff.calculate_krippendorffs_alpha_for_df(df,experiment_col='items',
                                                 annotator_col='annotators',
                                                 class_col='ratings')
print("Krippendorff's alpha: ", a)

Krippendorff's alpha:  0.10162601626016254


In [ ]:
# add a new column and recalculate alpha using that column here after collapsing  {-2,-1} to -1, and {1,2} to 1
df.loc[:, 'ratings_3'] = df['ratings']
df.replace({'ratings_3': {-2: -1, 2: 1}}, inplace=True)
df.head()


,annotators,items,ratings,ratings_3
0,1,1,-2,-1
1,2,1,-1,-1
2,3,1,-1,-1
3,1,2,1,1
4,2,2,2,1


In [ ]:
#calculate Krippendorff's alpha
b = simpledorff.calculate_krippendorffs_alpha_for_df(df,experiment_col='items',
                                                 annotator_col='annotators',
                                                 class_col='ratings_3')

print("Krippendorff's alpha: ", b)

Krippendorff's alpha:  0.6179775280898876


**Answer here**
It is significant in that the result number implies high reliability of the annotations. Therefore Reviewer 2 would say to collapse the annotated scores. However, in real-world, it is firmly discussed whether each labeled score is revisable.

In [ ]:
# calculate entropy here
check_arr = Counter(df.loc[:,'ratings'].to_numpy())
print("Entropy: ", entropy_from_counts(check_arr))


Entropy:  2.155221528859512


## Problem 8 : inter-annotator agreement 3 ( points)

The following arrays represent three annotators with four 5-point Likert-scale questions. All annotators "disagree" to varying levels on all four questions. Calculate Krippendorff's alpha. Discuss whether the score seems reasonable given your intuitive sense of agreement.

Now imagine you reverse the polarity of questions 2 and 4 by changing the wording, for example changing "burritos are sandwiches" to "burritos are *not* sandwiches". Assume the annotators respond in the same way, but with *positive* scores (ie flip the negative sign). Recalculate Krippendorff's alpha. Discuss why the answer does or does not change and whether this difference makes sense.

In [ ]:
annotators = [1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3]
items = [1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4]
ratings = [-2, -1, -1, -1, -1, -1, -2, -2, -1, -1, -1, -1]

# create dataframe and calculate alpha here
df2 = pd.DataFrame({"annotators": annotators, "items": items, "ratings": ratings})
c = simpledorff.calculate_krippendorffs_alpha_for_df(df2,experiment_col='items',
                                                 annotator_col='annotators',
                                                 class_col='ratings')

print("Krippendorff's alpha: ", c)

Krippendorff's alpha:  0.18518518518518523


In [ ]:
df2

,annotators,items,ratings
0,1,1,-2
1,2,1,-1
2,3,1,-1
3,1,2,-1
4,2,2,-1
5,3,2,-1
6,1,3,-2
7,2,3,-2
8,3,3,-1
9,1,4,-1


**Answer here**

When Krippendorff's alpha result is close to zero, it shows less agreement. This is different from what I expected in that the two items got same ratings and other two items also show similar inclination for ratings.

In [ ]:
# swap the polarity of questions 2 and 4 here, and recalculate alpha
df2.loc[:, 'ratings_polarity'] = df2['ratings']
df2.loc[((df2['items'] == 2) | (df2['items'] == 4)) & (df2['ratings'] == -1), 'ratings_polarity'] = 1
df2.loc[((df2['items'] == 2) | (df2['items'] == 4)) & (df2['ratings'] == -2), 'ratings_polarity'] = 2

df2

,annotators,items,ratings,ratings_polarity
0,1,1,-2,-2
1,2,1,-1,-1
2,3,1,-1,-1
3,1,2,-1,1
4,2,2,-1,1
5,3,2,-1,1
6,1,3,-2,-2
7,2,3,-2,-2
8,3,3,-1,-1
9,1,4,-1,1


In [ ]:
simpledorff.calculate_krippendorffs_alpha_for_df(df2,experiment_col='items',
                                                 annotator_col='annotators',
                                                 class_col='ratings_polarity')

0.5111111111111111

**Answer here**

Discuss why the answer does or does not change and whether this difference makes sense.

- The result has changed to an increase of krippendorff's alpha. The possible reason is that the changes of the score may affect disagreement estimation. In the first table, the ratings were only '-1, -2' which have less variance so that the expected disagreement would be higher that the latter table.
